In [1]:
import tensorflow as tf
import pandas as pd
import numpy
import matplotlib.pyplot as plt


In [2]:
hello = tf.constant("Hello, TensorFlow!")
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [3]:
df = pd.read_csv('data/data_train.csv')
df.head()

,Input_current,Output_voltage,Input_voltage,Duty_Cycle,Power,Irradiance,Temperature
0,6.818701,58.373037,29.782506,0.485,212.117783,1000,25
1,7.313437,53.815293,23.990136,0.557,180.751131,1000,60
2,6.056522,49.442379,24.298210,0.505,152.417456,836,60
3,5.735578,50.894729,26.911060,0.475,161.503052,836,48
4,5.376322,48.436413,26.198545,0.457,146.294526,755,48


Separamos as 4 entradas: Irradiance, Temperature, Tensão de Entrada e Corrente de entrada
Separamos a saida: DutyCycle

In [4]:
df_x = df.loc[:,['Irradiance','Temperature','Input_current','Input_voltage']] #Seleciona por nome
df_y = df.loc[:,['Duty_Cycle']] #Seleciona por nome
print(df_x[:5])
print(df_y[:5])
print(df_x.shape)

   Irradiance  Temperature  Input_current  Input_voltage
0        1000           25       6.818701      29.782506
1        1000           60       7.313437      23.990136
2         836           60       6.056522      24.298210
3         836           48       5.735578      26.911060
4         755           48       5.376322      26.198545
   Duty_Cycle
0       0.485
1       0.557
2       0.505
3       0.475
4       0.457
(2093, 4)


In [5]:
x=df_x.as_matrix()
y=df_y.as_matrix()
print(x.shape,y.shape)

(2093, 4) (2093, 1)


C:\Users\Jose\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\Jose\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [7]:
""" Neural Network.
A 2-Hidden Layers Fully Connected Neural Network (a.k.a Multilayer Perceptron)
implementation with TensorFlow. This example is using the MNIST database
of handwritten digits (http://yann.lecun.com/exdb/mnist/).
This example is using TensorFlow layers, see 'neural_network_raw' example for
a raw implementation with variables.
Links:
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf

# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)


# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer


# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    # Build the neural network
    logits = neural_net(features)

    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002811742D0B8>, '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\Jose\\AppData\\Local\\Temp\\tmptc1j0x7t', '_master': '', '_num_ps_replicas': 0, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_service': None, '_task_id': 0, '_tf_random_seed': None, '_is_chief': True, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_task_type': 'worker'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverH